In [17]:
import pandas as pd

In [18]:
#Station Data - map stations to zip code via city/landmark
#Creating Station Frame that contains all three files
file_names = ["201402_station_data.csv", "201408_station_data.csv", "201508_station_data.csv"]
sf_station_frame = []
for file in file_names:
    print(file)
    df = pd.read_csv(file)
    sf_station_frame.append(df)
sf_station_frames = pd.concat(sf_station_frame)
    
#Creating City to Zip Map
station_zip_dict = {'Station Zip' : ["94041", "94063", "94107", "94301", "95113"],\
                   'landmark' : ["Mountain View", "Redwood City", "San Francisco", "Palo Alto", "San Jose"]}
station_zip_map = pd.DataFrame(station_zip_dict)

#Merging two files so that we append zip
sf_station_data = pd.merge(sf_station_frames, station_zip_map, how='left', on="landmark")

#Removing data that isn't needed
del sf_station_data["lat"]
del sf_station_data["long"]
del sf_station_data["dockcount"]
del sf_station_data["installation"]
#del sf_station_data["landmark"]
del sf_station_data["name"]

#renaming station_id column so that we can merge it with trip data
sf_station_data.rename(columns={"station_id":"start station id", "name":"Name from Station File"}, inplace=True)

print(sf_station_data.shape)
sf_station_data.drop_duplicates(inplace = True)
print(sf_station_data.shape)
sf_station_data.head(72)

201402_station_data.csv
201408_station_data.csv
201508_station_data.csv
(209, 3)
(70, 3)


,start station id,landmark,Station Zip
0,2,San Jose,95113
1,3,San Jose,95113
2,4,San Jose,95113
3,5,San Jose,95113
4,6,San Jose,95113
5,7,San Jose,95113
6,8,San Jose,95113
7,9,San Jose,95113
8,10,San Jose,95113
9,11,San Jose,95113


In [19]:
#SF Data - column names have been renamed
#concatinate all files into one
file_names = ["201402_trip_data.csv", "201408_trip_data.csv", "201508_trip_data.csv"]
frames = []
for file in file_names:
    print(file)
    frame = pd.read_csv(file, parse_dates =["Start Date", "End Date"])
    #deleting columns that aren't needed
    del frame['Trip ID']
    del frame['Zip Code']
    frame.rename(columns={"Duration":"tripduration", "Start Date":"starttime", "End Date":"stoptime", \
                        "Start Terminal":"start station id","Start Station":"start station name",\
                        "End Terminal":"end station id", "End Station":"end station name", \
                        "Subscriber Type":"usertype", "Bike #":"bikeid"}, inplace=True)
    frames.append(frame)
print("Done")
sf_frame = pd.concat(frames)

#adding a city column

print(sf_frame.shape)
sf_frame.head()

201402_trip_data.csv
201408_trip_data.csv
201508_trip_data.csv
Done
(669959, 9)


,tripduration,starttime,start station name,start station id,stoptime,end station name,end station id,bikeid,usertype
0,63,2013-08-29 14:13:00,South Van Ness at Market,66,2013-08-29 14:14:00,South Van Ness at Market,66,520,Subscriber
1,70,2013-08-29 14:42:00,San Jose City Hall,10,2013-08-29 14:43:00,San Jose City Hall,10,661,Subscriber
2,71,2013-08-29 10:16:00,Mountain View City Hall,27,2013-08-29 10:17:00,Mountain View City Hall,27,48,Subscriber
3,77,2013-08-29 11:29:00,San Jose City Hall,10,2013-08-29 11:30:00,San Jose City Hall,10,26,Subscriber
4,83,2013-08-29 12:02:00,South Van Ness at Market,66,2013-08-29 12:04:00,Market at 10th,67,319,Subscriber


In [20]:
#Merge Trip with Station Data to include Zip from station, not Zip from Customer
print("started")
sf_frame = pd.merge(sf_frame, sf_station_data, how='left', on="start station id")
print("ended 1")
sf_frame["weather key"] = sf_frame["starttime"].dt.strftime('%Y%m%d') + sf_frame['Station Zip'].astype(str)
print("ended 2")

sf_frame.head()

started
ended 1
ended 2


,tripduration,starttime,start station name,start station id,stoptime,end station name,end station id,bikeid,usertype,landmark,Station Zip,weather key
0,63,2013-08-29 14:13:00,South Van Ness at Market,66,2013-08-29 14:14:00,South Van Ness at Market,66,520,Subscriber,San Francisco,94107,2013082994107
1,70,2013-08-29 14:42:00,San Jose City Hall,10,2013-08-29 14:43:00,San Jose City Hall,10,661,Subscriber,San Jose,95113,2013082995113
2,71,2013-08-29 10:16:00,Mountain View City Hall,27,2013-08-29 10:17:00,Mountain View City Hall,27,48,Subscriber,Mountain View,94041,2013082994041
3,77,2013-08-29 11:29:00,San Jose City Hall,10,2013-08-29 11:30:00,San Jose City Hall,10,26,Subscriber,San Jose,95113,2013082995113
4,83,2013-08-29 12:02:00,South Van Ness at Market,66,2013-08-29 12:04:00,Market at 10th,67,319,Subscriber,San Francisco,94107,2013082994107


In [21]:
#Weather Data from SF
file_names = ["201402_weather_data.csv", "201408_weather_data.csv", "201508_weather_data.csv"]
sf_weather_frame = []
for file in file_names:
    print(file)
    df = pd.read_csv(file, parse_dates =["PDT"])
    sf_weather_frame.append(df)

sf_weather_frames = pd.concat(sf_weather_frame)
del sf_weather_frames['Mean TemperatureF']
del sf_weather_frames['Max Dew PointF']
del sf_weather_frames['MeanDew PointF']
del sf_weather_frames['Min DewpointF']
del sf_weather_frames['Max Humidity']
del sf_weather_frames[' Mean Humidity']
del sf_weather_frames[' Min Humidity']
del sf_weather_frames[' Max Sea Level PressureIn']
del sf_weather_frames[' Mean Sea Level PressureIn']
del sf_weather_frames[' Min Sea Level PressureIn']
del sf_weather_frames[' Max VisibilityMiles']
del sf_weather_frames[' Mean VisibilityMiles']
del sf_weather_frames[' Min VisibilityMiles']
del sf_weather_frames[' Max Wind SpeedMPH']
del sf_weather_frames[' Max Gust SpeedMPH'] 
del sf_weather_frames[' Events']
del sf_weather_frames[' WindDirDegrees']

sf_weather_frames.rename(columns={" Mean Wind SpeedMPH":"avg wind", "Max TemperatureF":"max temp", \
                   "Min TemperatureF":"min temp", 'PrecipitationIn':"Pecipitation", \
                   "PDT":"Date from City File"}, inplace=True)

sf_weather_frames["weather key"] = sf_weather_frames["Date from City File"].dt.strftime('%Y%m%d') + sf_weather_frames['Zip'].astype(str)
sf_weather_frames.head()

201402_weather_data.csv
201408_weather_data.csv
201508_weather_data.csv


,Date from City File,max temp,min temp,avg wind,Pecipitation,CloudCover,Zip,weather key
0,2013-08-29,74.0,61.0,11.0,0,4.0,94107,2013082994107
1,2013-08-30,78.0,60.0,13.0,0,2.0,94107,2013083094107
2,2013-08-31,71.0,57.0,15.0,0,4.0,94107,2013083194107
3,2013-09-01,74.0,58.0,13.0,0,4.0,94107,2013090194107
4,2013-09-02,75.0,62.0,12.0,0,6.0,94107,2013090294107


In [30]:
#merging all sf data, saving to csv file
sf_merged_data = pd.merge(sf_frame, sf_weather_frames, how='left', on="weather key")
sf_merged_data.to_csv(path_or_buf="sf_merged_data.csv")
print("done")
del sf_merged_data["Date from City File"]
del sf_merged_data["Station Zip"]
del sf_merged_data["Zip"]
del sf_merged_data[" CloudCover"]
sf_merged_data.rename(columns={"landmark":"city"}, inplace=True)
print("done1")
sf_merged_data.head()

done
done1


,tripduration,starttime,start station name,start station id,stoptime,end station name,end station id,bikeid,usertype,city,weather key,max temp,min temp,avg wind,Pecipitation
0,63,2013-08-29 14:13:00,South Van Ness at Market,66,2013-08-29 14:14:00,South Van Ness at Market,66,520,Subscriber,San Francisco,2013082994107,74.0,61.0,11.0,0
1,70,2013-08-29 14:42:00,San Jose City Hall,10,2013-08-29 14:43:00,San Jose City Hall,10,661,Subscriber,San Jose,2013082995113,81.0,63.0,7.0,0
2,71,2013-08-29 10:16:00,Mountain View City Hall,27,2013-08-29 10:17:00,Mountain View City Hall,27,48,Subscriber,Mountain View,2013082994041,80.0,64.0,5.0,0
3,77,2013-08-29 11:29:00,San Jose City Hall,10,2013-08-29 11:30:00,San Jose City Hall,10,26,Subscriber,San Jose,2013082995113,81.0,63.0,7.0,0
4,83,2013-08-29 12:02:00,South Van Ness at Market,66,2013-08-29 12:04:00,Market at 10th,67,319,Subscriber,San Francisco,2013082994107,74.0,61.0,11.0,0


In [31]:
sf_merged_data.shape

(669959, 15)

In [8]:
#NYC Data - column names have been renamed
#concatinate all files into one
file_names = [year + "-" + month + "-citibike-tripdata.csv" for year in ["2013", "2014", "2015", "2016"] for month in ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]]
file_names = file_names[6:-6]
#file_names = file_names[6:8]
frames = []
for file in file_names:
    print(file)
    frame = pd.read_csv(file, parse_dates =["starttime", "stoptime"])
    #deleting columns that aren't needed
    del frame['start station latitude']
    del frame['start station longitude']
    del frame['end station latitude']
    del frame['end station longitude']
    del frame['birth year']
    del frame['gender']
    frames.append(frame)
print("Done")
nyc_frame = pd.concat(frames)
nyc_frame["city"] = "nyc"

#creating a weather key
nyc_frame["weather key"] = nyc_frame["starttime"].dt.strftime('%Y%m%d')

print(nyc_frame.shape)
nyc_frame.head()
        

2013-07-citibike-tripdata.csv
2013-08-citibike-tripdata.csv
2013-09-citibike-tripdata.csv
2013-10-citibike-tripdata.csv
2013-11-citibike-tripdata.csv
2013-12-citibike-tripdata.csv
2014-01-citibike-tripdata.csv
2014-02-citibike-tripdata.csv
2014-03-citibike-tripdata.csv
2014-04-citibike-tripdata.csv
2014-05-citibike-tripdata.csv
2014-06-citibike-tripdata.csv
2014-07-citibike-tripdata.csv
2014-08-citibike-tripdata.csv
2014-09-citibike-tripdata.csv
2014-10-citibike-tripdata.csv
2014-11-citibike-tripdata.csv
2014-12-citibike-tripdata.csv
2015-01-citibike-tripdata.csv
2015-02-citibike-tripdata.csv
2015-03-citibike-tripdata.csv
2015-04-citibike-tripdata.csv
2015-05-citibike-tripdata.csv
2015-06-citibike-tripdata.csv
2015-07-citibike-tripdata.csv
2015-08-citibike-tripdata.csv
2015-09-citibike-tripdata.csv
2015-10-citibike-tripdata.csv
2015-11-citibike-tripdata.csv
2015-12-citibike-tripdata.csv
2016-01-citibike-tripdata.csv
2016-02-citibike-tripdata.csv
2016-03-citibike-tripdata.csv
2016-04-ci

,tripduration,starttime,stoptime,start station id,start station name,end station id,end station name,bikeid,usertype,city,weather key
0,634,2013-07-01 00:00:00,2013-07-01 00:10:34,164,E 47 St & 2 Ave,504,1 Ave & E 15 St,16950,Customer,nyc,20130701
1,1547,2013-07-01 00:00:02,2013-07-01 00:25:49,388,W 26 St & 10 Ave,459,W 20 St & 11 Ave,19816,Customer,nyc,20130701
2,178,2013-07-01 00:01:04,2013-07-01 00:04:02,293,Lafayette St & E 8 St,237,E 11 St & 2 Ave,14548,Subscriber,nyc,20130701
3,1580,2013-07-01 00:01:06,2013-07-01 00:27:26,531,Forsyth St & Broome St,499,Broadway & W 60 St,16063,Customer,nyc,20130701
4,757,2013-07-01 00:01:10,2013-07-01 00:13:47,382,University Pl & E 14 St,410,Suffolk St & Stanton St,19213,Subscriber,nyc,20130701


In [11]:
nyc_frame.to_csv(path_or_buf="nyc_frame.csv")

In [12]:
#Weather Data from GOV - missing data from earlier days, only starts from '20130901'
#Read in data and format weather key (date only for NYC)
print("hello")
nyc_weather_frames = pd.read_csv("NYC Weather Data.csv", parse_dates = ["DATE"], dtype = 'str')
nyc_weather_frames["weather key"] = nyc_weather_frames["DATE"].dt.strftime('%Y%m%d')
#Deleting columns not needed
del nyc_weather_frames['STATION']
del nyc_weather_frames['ELEVATION']
del nyc_weather_frames['LATITUDE']
del nyc_weather_frames['LONGITUDE']
#del nyc_weather_frames['DATE']
del nyc_weather_frames['MDPR']
del nyc_weather_frames['DAPR']
del nyc_weather_frames['SNWD']
del nyc_weather_frames['SNOW']
del nyc_weather_frames['TAVG']
del nyc_weather_frames['TOBS']
del nyc_weather_frames['WDF2']
del nyc_weather_frames['WDF5']
del nyc_weather_frames['WSF2']
del nyc_weather_frames['WSF5']
del nyc_weather_frames['PGTM']
del nyc_weather_frames['WT09']
del nyc_weather_frames['WT01']
del nyc_weather_frames['WT06']
del nyc_weather_frames['WT05']
del nyc_weather_frames['WT02']
del nyc_weather_frames['WT11']
del nyc_weather_frames['WT04']
del nyc_weather_frames['WT08']
del nyc_weather_frames['WT03']
del nyc_weather_frames['WT10']
#rename columns
nyc_weather_frames.rename(columns={"AWND":"avg wind", "TMAX":"max temp", "TMIN":"min temp", \
                        "PRCP":"Pecipitation", "DATE":"Date from Weather File"}, inplace=True)

#filter out stations that aren't Central Park: 'NEW YORK CENTRAL PARK OBS BELVEDERE TOWER NY US'
nyc_weather_frames = nyc_weather_frames[nyc_weather_frames.STATION_NAME == 'NEW YORK CENTRAL PARK OBS BELVEDERE TOWER NY US']
del nyc_weather_frames['STATION_NAME']

nyc_weather_frames.head()


,Date from Weather File,Pecipitation,max temp,min temp,avg wind,weather key
51794,2013-09-01,0.00,84,75,2.5,20130901
51795,2013-09-02,0.05,82,74,0.9,20130902
51796,2013-09-03,0.03,82,68,2.9,20130903
51797,2013-09-04,0.00,82,65,5.4,20130904
51798,2013-09-05,0.00,80,64,5.6,20130905


In [13]:
nyc_weather_frames.to_csv(path_or_buf="nyc_weather_frames.csv")

In [14]:
#merging NYC Data
#For datetime64[ns] types, NaT represents missing values, missing weather data
print("start")
nyc_merged_data = pd.merge(nyc_frame, nyc_weather_frames, how='left', on="weather key")
#nyc_merged_data.describe
nyc_merged_data.head()
nyc_merged_data.to_csv(path_or_buf="nyc_merged_data.csv")
print("done")

start
done


In [16]:
nyc_merged_data.shape

(28732390, 16)

In [ ]:
#SF Data - Figuring out date columns 
print("start")
sample_frame = pd.read_csv("201402_trip_data.csv", parse_dates =["Start Date", "End Date"])
print("end")
#sample_frame["time elapsed"] =  sample_frame["End Date"] - sample_frame["Start Date"]
print(sample_frame.shape)
print("start1")
sample_frame["Date"] = sample_frame["Start Date"].dt.strftime('%Y%m%d')
print("end1")
sample_frame.head()
